<a href="https://colab.research.google.com/github/laura-turnbull-lloyd/IGR_remote_sensing/blob/main/IGR_2025_26_WS04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IGR Workshop 4: Using remote sensing to detect surface changes over time
_Laura Turnbull-Lloyd, IGR, 2026._

# Overview
In the session last week, you explored the structure of a Landsat image, had a go at visualizing different band combinations, and then you calculated 3 indices:
*  Normalized Difference Vegetation Index (NDVI)
*  Enhanced Vegetation Index (EVI)
*  Normalized Difference Moisture Index (NDMI)

You then undertook some preliminary analysis to see how these indices varied for different surface cover types that you mapped out in the field.

This week, we are going to build on the work you undertook last week, to use a time sequence of Landsat images to explore changes in surface characteristics over both space and time, really delving into the power that remote sensing offers.

This week we will build directly on that foundation by adding a temporal dimension. Using a time sequence of Landsat images, you will explore how surface characteristics change throughout the year and across the Langdon Beck catchment, and you will see how the conclusions you draw depend on data quality, especially cloud and shadow contamination.

By the end of the worksheet you should be able to:

* create and interpret a monthly EVI time series
* link patterns in EVI to what you observed in the field (e.g., vegetation condition, land cover, seasonality)
* recognise when apparent "change" is actually an artefact of clouds, snow, or limited observations, and understand why it is important to pay attention to data quality indicators


# Data you will work with
Last week you worked with Landsat Level 2 surface reflectance data, and from this you calculated the Enhanced Vegetation Index, which is generally considered to be better for assessing vegetation health than the Normalised Difference Vegetation Index, [Heute et al, 2002](https://doi.org/10.1016/S0034-4257(02)00096-2)).

This week, we will focus on temporal variations in the EVI. You already know how to calculate the EVI from Landsat Surface Reflectance products using the following equation.
$$
\mathrm{EVI} = G \, \frac{\mathrm{NIR} - \mathrm{Red}}{\mathrm{NIR} + C_1\times\mathrm{Red} - C_2\times\mathrm{Blue} + L}
$$

This week, to simplify things, you have been provided with the EVI already calculated for you, for nine mostly cloud free images during 2025.

The EVI was calculated for you on a platform called [Google Earth Engine](https://earthengine.google.com/) (you don't need to worry about this for now, but if you're interested in exploring remote sensing further, you might like to take a look!).

If you recall that the Landsat repeat cycle is 16 days, you might be wondering why there are only nine images for you to work with. The answer is mostly clouds!

As noted by [NASA](https://science.nasa.gov/earth/earth-observatory/measuring-vegetation-ndvi-evi/), clouds and aerosols can often block the satellites view of the surface entirely, glare from the sun can saturate certain pixels, and temporary malfunctions in the satellite instruments themselves can distort an image. Consequently, many of the pixels in a day's worth of images are indecipherable, and maps made from the daily Vegetation Indices are patchy at best.

One simple approach is to filter the image collection using cloud-cover information, keeping only scenes with relatively clear conditions. In 2025 we have ended up with only 9 images, because only 9 scenes had less than 20% cloud cover within the window that covers Langdon Beck.

These images are provided for you as GeoTIFFS (rasters) - one file for each time stamp. The image acquisition date is recorded in the file name. You are also provided with the cloud cover infromation (processed into a simple cloud mask), also provided by Landsat, to enable you to see clearly where the cloud cover occurs for the slightly cloudier images.

With these data, you can explore changes in vegetation condition (linked to vegetation phenology) over time within Langdon Beck. The diagram below summarised how we can take the remote sensing EVI product, and from it be able to say something meaningful about changes in vegetation condition over time.

<div align="center">
<img src='https://github.com/laura-turnbull-lloyd/IGR_remote_sensing/blob/main/Figures/data-to-insight.png?raw=true' width='75%' alt='data-to-insight.png'>
</div>


# Getting started
As with last week, we will use the same core packages and functions to download, read in and process the data. These are **matplotlib**, **rasterio** etc.

The data are stored for you on a GitHub repository, for easy access, and within this worksheet you will download the data to your google drive, and then work with it from there.



# Questions!
There are questions throughout the worksheet. You can answer these questions on the microsoft form. This is just for you to check your progress (I won't be looking at any marks). The key thing is that you compare your answer to the suggested answer (don't worry if it says you're wrong if you've given the right answer - it might just be a formatting issue).

The link to the questions is [here](https://forms.office.com/e/405JVzXhzL).

In [ ]:
#@title First, you need to run this block to install packages and load libraries.

!pip -q install "geopandas>=0.14"            # First we install the geopandas package as it isn't a default package in colab
import geopandas as gpd                      # Then we load the package - needed for working with the catchment shapefile
import matplotlib.pyplot as plt
import os
import re
import os
import re
import pandas as pd                          # the main packges for working with data
import numpy as np
import math
import rasterio                              # rasterio is the main package we're using for working with the raster data
from rasterio.features import geometry_mask
from rasterio.windows import from_bounds
from rasterio.windows import Window
from rasterio.windows import bounds as window_bounds
from rasterio.mask import mask
from rasterio.features import rasterize
from rasterio.features import geometry_mask
import glob
import zipfile                               # this and the other libraries below are used to import the data from github via a web link (url)
from urllib.parse import urlparse
import requests
from tqdm import tqdm
import requests, io

## Question: What is the main package we'll be using to process the raster data?

Then, you can download the data (in a zipped up folder) and then extract the files to your google drive. Don't worry about the detail here - this is just getting the files set up for you.

In [ ]:
#@title Run this block to download the ZIP and extract all files into a folder

# USER SETTINGS
ZIP_URL = "https://github.com/laura-turnbull-lloyd/IGR_remote_sensing/raw/refs/heads/main/Data/L8_EVI_cloudfiltered-20260210T171611Z-1-001.zip"

DOWNLOAD_DIR = "/content/downloads"                 # zip file saved here
OUT_DIR      = "/content/L9_EVI_cloudfiltered_files" # ALL extracted files go here (single folder)

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# FUNCTION TO DOWNLOAD THE DATA
def download_file(url, filename):
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()
    total = int(r.headers.get("content-length", 0)) or None
    chunk = 1024 * 1024  # 1 MB
    with open(filename, "wb") as f:
        with tqdm(total=total, unit="B", unit_scale=True, desc=os.path.basename(filename)) as pbar:
            for data in r.iter_content(chunk):
                if data:
                    f.write(data)
                    pbar.update(len(data))

# DOWNLOADING THE ZIPPED UP DATA
zip_name = os.path.basename(urlparse(ZIP_URL).path)
zip_path = os.path.join(DOWNLOAD_DIR, zip_name)

print("Downloading ZIP...")
download_file(ZIP_URL, zip_path)
print("ZIP downloaded:", zip_path)

# EXTRACTING THE ZIP CONTENTS INTO OUT_DIR
print("\nExtracting files to:", OUT_DIR)

with zipfile.ZipFile(zip_path, "r") as z:
    members = [m for m in z.namelist() if not m.endswith("/")]  # files only
    for m in tqdm(members, desc="Extracting", unit="file"):
        # Force everything into a single folder by stripping subfolders
        base = os.path.basename(m)
        if base == "":
            continue

        out_path = os.path.join(OUT_DIR, base)

        # Extract file bytes and write to OUT_DIR
        with z.open(m) as src, open(out_path, "wb") as dst:
            dst.write(src.read())

print("\nDone!")
print("Extracted file count:", len([f for f in os.listdir(OUT_DIR) if os.path.isfile(os.path.join(OUT_DIR, f))]))
print("First 10 extracted files:")
for f in sorted(os.listdir(OUT_DIR))[:10]:
    print(" -", f)



You should see in the text above that the files have successfully been downloaded and extracted.

# Extracting the dates of each image
To be able to explore how vegetation condition over time, we need to know the date represented by each image. Conveniently, the data is given in each file name, along with some other useful infromation, e.g.

L9_20250309_cloud0pct_EVI.tif.

This tells us the Landsat sensor (L9), the date in the format YYYYMMDD, the cloud cover as a %, and what the data are (EVI or cloud mask).

So in the above example, it's the EVI calculated for 9th March 2025 and there's 0% cloud cover.

## Question: Have a look through the files you've extracted and stored in a folder called "L9_EVI_cloudfiltered_files". What's the highest cloud cover in all these files?
Tip: to view the files in this folder, click on the folder icon in the left menu bar, and click on the "L9_EVI...." folder to view its contents.

We can write a code to extract the date from the file name, as well as the type of file (EVI or cloud mask), and ultimately create a list of all the EVI and cloud mask files, along with their corresponding dates.

Let's have a go...

In [ ]:
#@title Extracting the date of each image and creating lists of EVI and cloud mask files

date_re = re.compile(r"(19|20)\d{2}(0[1-9]|1[0-2])(0[1-9]|[12]\d|3[01])")
# Here, re.compile turns the pattern into a reusable “date finder” that we can re-use.
# Broken down:
# (19|20)\d{2} matches a 4-digit year from 1900–2099: 19 or 20, followed by two digits (\d{2})
# (0[1-9]|1[0-2]) matches a month 01–12: 01–09 or 10–12
# (0[1-9]|[12]\d|3[01]) matches a day 01–31: 01–09, 10–29, 30–31

# Function to identify if the file is an EVI file or a cloud mask file.
def classify_file(name: str):
    """Return 'evi', 'cloudmask', or None based on filename."""
    base = os.path.basename(name).lower()
    if "cloudmask" in base or "cloud_mask" in base: # this picks out if the file name contains cloud mask
        return "cloudmask"
    if base.endswith("_evi.tif") or "evi" in base: # this picks out if the file name contains EVI
        return "evi"
    return None

evi_records, cloudmask_records = [], [] # This line creates two empty Python lists in one go where we
                                        # will store the dates.

# List only the top-level files in OUT_DIR
for fn in os.listdir(OUT_DIR):                        # This loop through every file in OUT_DIR
    if not fn.lower().endswith((".tif", ".tiff")):    # This ignores any files that aren't tif files
        continue

    full_path = os.path.join(OUT_DIR, fn)             # This build the full path (the directory and the filename)

    date_m = date_re.search(fn)                       # This extracts the date from the filename
    date_str = date_m.group(0) if date_m else None    # This extracts a date from the filename and assigns it to date_str
    kind = classify_file(fn)                          # This classifies the file as "evi" or "cloudmask"

    rec = {"path": full_path, "filename": fn, "date": date_str, "kind": kind} # This creates a "record" dictionary storing the useful info about the files.

    if kind == "evi":                                 # This appends the record to the right list
      evi_records.append(rec)
    elif kind == "cloudmask":
      cloudmask_records.append(rec)

print("EVI files:", len(evi_records))                 # Finally, this prints out the number of EVI files
print("Cloudmask files:", len(cloudmask_records))     # and this prints out the number of cloudmask files.

# Sort by date (strings YYYYMMDD sort correctly) (important for later plotting of the time series)
evi_records.sort(key=lambda r: (r["date"] is None, r["date"], r["filename"]))
cloudmask_records.sort(key=lambda r: (r["date"] is None, r["date"], r["filename"]))

print("\nFirst 5 EVI records:")
for r in evi_records[:5]:
    print(r["date"], r["filename"])


## Question: What's the date of the first EVI file in this data collection for 2025? Give your answer in the format _yyyymmdd_.

# Data pre-processing
Here, you're going to create a list of dates and file paths (i.e. the location where the file is stored along with the fielname) to read in, in a format that the rasterio package can make sense of, and then extract the spatial information needed for plotting and cropping the data.

In [ ]:
#@title Reading the data into a raster with rasterio

# Create a list of dates and paths
dates = [r["date"] for r in evi_records]
evi_paths = [r["path"] for r in evi_records]
cloud_paths = [r["path"] for r in cloudmask_records]

# Open the first EVI raster to get bounds/CRS/transform + extent for plotting
with rasterio.open(evi_paths[0]) as LB:  # This opens the first file and stores it in "LB"
    bounds = LB.bounds                   # These are all parameters that make sure the data plots correctly on the right coordinates!
    landsat_crs = LB.crs
    transform = LB.transform
    nodata0 = LB.nodata
    height0, width0 = LB.height, LB.width

extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

print("\nRaster metadata (from first EVI file):")       # This just prints out the spatial information below so you can check it
print(" Bounds:", bounds)
print(" Extent:", extent)
print(" CRS:", landsat_crs)
print(" Transform:", transform)
print(" Shape (height, width):", (height0, width0))
print(" NoData:", nodata0)

# Read & stack EVI and cloudmask into aligned 3D arrays: (time, y, x)
EVI_list = []
CLOUD_list = []

# This block opens all the data and creates raster stacks for the EVI cloud data and the cloud data.
for evi_p, cloud_p in zip(evi_paths, cloud_paths):
    # --- Read EVI ---
    with rasterio.open(evi_p) as src:
        evi_arr = src.read(1).astype("float32")
        if src.nodata is not None:
            evi_arr[evi_arr == src.nodata] = np.nan
        EVI_list.append(evi_arr)

    # --- Read cloud mask ---
    with rasterio.open(cloud_p) as src:
        cloud_arr = src.read(1).astype("float32")
        # Cloud masks are typically 0/1; convert nodata to NaN if present
        if src.nodata is not None:
            cloud_arr[cloud_arr == src.nodata] = np.nan
        CLOUD_list.append(cloud_arr)

EVI_stack = np.stack(EVI_list, axis=0)      # (time, y, x)
CLOUD_stack = np.stack(CLOUD_list, axis=0)  # (time, y, x)

print("\Dimensions of the data:")
print(" EVI_stack shape   (time, y, x):", EVI_stack.shape)
print(" CLOUD_stack shape (time, y, x):", CLOUD_stack.shape)


After all that data pre-processing we're now in a position to do some more interesting stuff with it.

As usual, we'll first have a little look at the data by plotting it out, looking at the EVI and cloud mask data side-by-side. The cloud mask data is very simple. Where there is a 1, it means there was a cloud, where there is a 0, there are no clouds.

So rather than refer to the layer in the raster stack we want to look at (we'd have to update this in a couple of places), we can set up a variable, _t_  with the layer number that we want to look at.

Have a go at viewing several layers representing the EVI and cloud cover at different dates (by changing the value of _t_). Pay attention to what happens to the calculated EVI values where there are clouds.

In [ ]:
#@title Plot the first layer of the EVI and cloudmask stacks, side by side

t = 0  # the layer you want to look at - you can change this! Remember that python indexing starts at 0, so timestep 1 is index 0.
dates_dt = [pd.to_datetime(d, format="%Y%m%d") for d in dates] # convert dates to a proper date format for plotting

plt.figure(figsize=(12, 5))

# --- EVI ---
plt.subplot(1, 2, 1)
plt.imshow(EVI_stack[t], extent=extent, origin="upper")
plt.colorbar(label="EVI")
plt.title(f"EVI (layer {t}, {dates_dt[t].date()})")             # This is set up so that the plot title will automatically take the date of that layer.
plt.xlabel("Easting")
plt.ylabel("Northing")
plt.locator_params(axis="x", nbins=4)                           # This plots out fewer x-axis tick labels

# --- Cloud mask ---
plt.subplot(1, 2, 2)
plt.imshow(CLOUD_stack[t], extent=extent, origin="upper")
plt.colorbar(label="Cloud mask (1 = cloud/shadow/snow)")
plt.title(f"Cloud mask (layer {t}, {dates_dt[t].date()})")      # This is set up so that the plot title will automatically take the date of that layer.
plt.xlabel("Easting")
plt.ylabel("Northing")
plt.locator_params(axis="x", nbins=4)

plt.tight_layout()
plt.show()


## Question: What happens to the EVI values in regions where there are clouds?

Now, before doing any further quantitative analysis on the EVI values, let's crop it to the catchment outline as you did last week (the code below is taken from the worksheet last week, and modified for this application).

---



In [ ]:
#@title First, read in data that's provided for you on Github
url = "https://github.com/laura-turnbull-lloyd/IGR_remote_sensing/raw/refs/heads/main/Data/nrfa_25011.zip"

# Download the zip file into memory
r = requests.get(url)                         # This downloads whatever is at that URL, in this case a zipped folder containing the catchment data.
z = zipfile.ZipFile(io.BytesIO(r.content))    # This processes the data so that Python can treat it like an in-memory file.

# read in the shapefile
z.extractall("nrfa_25011")                    # This extracts all files from the ZIP into a folder called nrfa_25011 in your current working directory.
LB_25011 = gpd.read_file("nrfa_25011")        # This reads in the catchment outline data using the geopandas package.
catchment = LB_25011.to_crs(landsat_crs)      # This puts the catchment outline in the same coordinate system as the Landsat data.

In [ ]:
#@title Now we can crop the data
geoms = [geom.__geo_interface__ for geom in catchment.geometry]

xmin, ymin, xmax, ymax = catchment.total_bounds

# Build a window and SNAP it to the pixel grid
win = from_bounds(xmin, ymin, xmax, ymax, transform=transform)
win = win.round_offsets().round_lengths()

# Crop stacks using the window (no int slicing)
EVI_crop   = EVI_stack[:, win.row_off:win.row_off+win.height,
                          win.col_off:win.col_off+win.width]
CLOUD_crop = CLOUD_stack[:, win.row_off:win.row_off+win.height,
                            win.col_off:win.col_off+win.width]

# Updated transform for the cropped arrays
transform_crop = rasterio.windows.transform(win, transform)

# Mask on the cropped grid
inside = geometry_mask(
    geoms,
    out_shape=(win.height, win.width),
    transform=transform_crop,
    invert=True  # True = inside geometry is True
)

EVI_masked = EVI_crop.astype("float32", copy=True)
CLOUD_masked = CLOUD_crop.astype("float32", copy=True)

EVI_masked[:, ~inside] = np.nan
CLOUD_masked[:, ~inside] = np.nan

b_crop = window_bounds(win, transform)  # (left, bottom, right, top)
left, bottom, right, top = b_crop


Now, let's plot out the EVI cropped data to make sure it all looks good...

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(EVI_masked[1],  origin="upper", extent=[left, right, bottom, top])
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_aspect("equal")
plt.show()





# Landsat Vegetation Phenology
We can now investigate changes in vegetation phenology using the Landsat 9-derived EVI, looking at how it varies over time (see figure below).

The EVI is a widely used indicator of vegetation greenness and condition that is designed to be less sensitive to atmospheric effects and background soil influences than simpler vegetation indices such as the NDVI.

<div align="center">
  <img src="https://raw.githubusercontent.com/laura-turnbull-lloyd/IGR_remote_sensing/main/Figures/Seasonal-dynamics-of-the-Enhanced-Vegetation-Index-EVI.png"
       width="75%"
       alt="Seasonal dynamics of the EVI">
</div>

Because the EVI data are stored in a raster stack, we can easily calculate simple summary statistics, such as the mean and median (more robust to outliers) to explore how the EVI changes over time.

Let's have a go at exploring vegetation condition over the Langdon Beck catchment over 2025.

As well as plotting out the data, we'll also export (to your google drive) the summary dataframe, containing the mean and median values for each date.

In [ ]:
#@title Summarise the mean and median EVI over time

# Median
evi_median = np.nanmedian(EVI_masked, axis=(1, 2))  # Here we're calculating the median across all pixels in the catchment (ignores NaNs)
                                                    # axis=(1, 2) tells np.nanmedian to take the median across the y and x dimensions
                                                    # (rows and columns) for each time step.
                                                    # If we were calculating the mean of a single image (without the temporal dimension)
                                                    # it would simply be np.nanmedian(EVI_masked)
# Mean
evi_mean = np.nanmean(EVI_masked, axis=(1, 2))      # And here we're doing the same, but this time calculating the mean

# Standard deviation
evi_std  = np.nanstd(EVI_masked, axis=(1, 2))

# Make sure dates is a pandas-friendly datetime index
t = pd.to_datetime(dates)

# Put into a table (a better structure for plotting out the time series)
summary = pd.DataFrame({
    "date": t,
    "median_evi": evi_median,
    "mean_evi": evi_mean
}).sort_values("date").reset_index(drop=True)

print(summary)
print("\nRows:", len(summary))


In [ ]:
#@title  Plot date against mean and median EVI

fig, ax = plt.subplots(figsize=(7,4))
ax.plot(summary["date"], summary["mean_evi"], marker="o", color="red",  label="Mean EVI")
ax.plot(summary["date"], summary["median_evi"], marker="o", color="blue", label="Median EVI")
ax.set_xlabel("Date")
ax.set_ylabel("EVI")
ax.set_title("Catchment mean & median EVI through time")
ax.legend()

fig.tight_layout()
plt.show()



In [ ]:
#@title Save the summary EVI values to CSV file (i.e. spreadsheet format)
out_csv = "/content/Landsat_evi_time_summary.csv"
summary.to_csv(out_csv, index=False)
print("\nSaved:", out_csv)

## Question: What was the highest median EVI for Langdon Beck during 2025?

# Compositing Landsat EVI measurements

Due to the fact that we can only do sensible EVI analysis when there's low cloud cover, we don't have any data points extending into the months at the end of the year, and sporadic measurements throughout summer. This is a limitation of EVI measurements using only a single Landsat image.

One approach to addressing this limitation in remote sensing is to create was called a composite image, where EVI values calculated for multiple different times are merged together, often taken maximum values, creating what's usually referred to as a maximum value composite. This type of approach allows us to make use of any cloud free pixels across a collection of partially cloudy images.

Often, monthly/seasonal maximum value composites are made using Landsat data, so that several images can be merged.

Here, we're going to have a go at creating a maximum value composite, for 2025 (seeing as we've already got a collection of files already prepped for 2025!).
We will combine all EVI images for the year and, for each pixel, keep the maximum EVI value observed across the time series. This approach is useful because it is likely to capture the years peak vegetation condition (peak greenness/biomass), while reducing the impact of issues like cloud contamination and seasonal snow (we'll come back to this point later).

A maximum-value annual composite doesn't tell us anything about within-year vegetation dynamics, but it does provide a robust way to compare year-to-year differences in vegetation condition (should we compare different years).


In [ ]:
#@title EVI maximum value composite calculation
EVI_max = np.nanmax(EVI_masked, axis=0)

Now, on your own, enter code in the code block below to plot out the EVI maximum value composite that you have just created. You may wish to make use of plotting code in this worksheet or the one from last week.

In [ ]:
#@title Enter your own code to plot out the EVI maximum value composite for 2025



Now, see if you can calculate the median value of this maximum value composite, to see how much it varies compared to the highest median value you reported before.

In [ ]:
#@title On your own, calculate the EVI median of the maximum value composite and print out your result

# Enter your code here.


## Question: What's the median value of the EVI maximum value composite?

# Exploring how vegetation condition/phenology varies between land cover types

Last week you explored how the NDVI and EVI varied in relation to the surface cover descriptions that you took in the field.

We're going to do a similar task now, but instead of using the surface cover data that represents only a few locations on your hillslope transects, this time we're going to use a land cover dataset (from 2023), which was generated by Centre for Ecology and Hydrology and can be accessed via DigiMap. Information about the dataset can be accessed [here](https://www.ceh.ac.uk/sites/default/files/2024-06/LCM2023ProductDocumentation.pdf). These data have been pre-processes, and clipped to the Langdon Beck catchment.  

In [ ]:
#@title Open landcover data
url = "https://github.com/laura-turnbull-lloyd/IGR_remote_sensing/raw/refs/heads/main/Data/landcover_landsat.zip"

# Download the zip file into memory
r = requests.get(url)                         # This downloads whatever is at that URL, in this case a zipped folder containing the catchment data.
z = zipfile.ZipFile(io.BytesIO(r.content))    # This processes the data so that Python can treat it like an in-memory file.

# read in the shapefile
z.extractall("landcover_landsat")                    # This extracts all files from the ZIP into a folder called Landcover_Langdon in your current working directory.
landcover = gpd.read_file("landcover_landsat")    # This reads in the landcover data using the geopandas package.
landcover_LB = landcover.to_crs(landsat_crs)

Let's check the land cover data plots in the correct place...

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(EVI_max,  origin="upper", extent=[left, right, bottom, top])
landcover_LB.boundary.plot(ax=ax, linewidth=1, color="red")             # here we're overlaygoverlaying the landcover data as an outline
ax.set_title("FCC with catchment boundary")
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
ax.set_aspect("equal")
plt.show()


That looks good, so let's now visualize the land cover classes. These are stored as numeric id's in an attribute called "_mode".

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

landcover_LB.plot(column="_mode", legend=True, ax=ax)

# move legend outside
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.02, 1))
leg.set_loc("upper left")

plt.tight_layout()
plt.show()

As you're familiar with the catchment, hopefully you have some idea which land cover types related to these numeric id's.

We can also map these numeric id's onto the the actual land cover description, which will make it a more useful map of land cover.

These numeric id's correspond to land cover as follows:

* 11 = Bog
* 4 = Improved grassland
* 7 = Acid grassland
* 12 = Inland rock
* 1 = Deciduous woodland
* 21 = Suburban
* 2 = Coniferous woodland
* 9 = Heather

So, we can alter the above code to give the actual land cover descriptions in the legend, to make the map more meaningful:

In [ ]:
#@title Plot out the landcover map again, this time with a more meaningful legend
lc_labels = {
    "11": "Bog",
    "4": "Improved grassland",
    "7": "Acid grassland",
    "12": "Inland rock",
    "1": "Deciduous woodland",
    "21": "Suburban",
    "2": "Coniferous woodland",
    "9": "Heather"
}

landcover_LB["lc_name"] = landcover_LB["_mode"].astype(str).map(lc_labels)

fig, ax = plt.subplots(figsize=(6, 6))

landcover_LB.plot(column="lc_name", cmap="tab10", legend=True, ax=ax)

# move legend outside
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.02, 1))
leg.set_loc("upper left")

plt.tight_layout()
plt.show()


Question: Does the "inland rock" tend to have a higher or lower EVI value?

# Zonal analysis
Now, we're going to do something called "zonal analysis", to summarize the mean EVI value for each land cover type at each point in time. This is potentially very valuable as it might enable us to make statements in relation to carbon storage, or how wet the landscape is (remember the tight correlation between the EVI and the normalized difference moisture index last week).

This code is quite complicated, but basically what it's doing is cropping the EVI data to each surface cover type, and then it calculates the mean EVI value in that surface cover type.

It repeats this for every surface cover type.

In [ ]:
T, H, W = EVI_masked.shape
t = pd.to_datetime(dates)
rows = []

for idx, row in landcover_LB.iterrows():     # This is looping through each surface cover type
    geom = row.geometry.__geo_interface__
    cls = row["_mode"]

    m = geometry_mask([geom], out_shape=(H, W), transform=transform_crop,
                      invert=True, all_touched=True)

    mean_ts = np.nanmean(EVI_masked[:, m], axis=1)          # This calcualtes the mean EVI

    for date, val in zip(t, mean_ts):                          # This summarizes the calculations for each date.
        rows.append({"poly_id": idx, "landcover": cls, "date": date, "mean_evi": val})

df = pd.DataFrame(rows)

# Summarise across polygons for each class and date
summary_evi = df.groupby(["landcover", "date"])["mean_evi"].mean().reset_index()


In [ ]:
#@title Let's see what the structure of the data looks like
print(summary_evi)

Let's now have a look to see if there's much of a difference in vegetation condition across the different land cover types.

In [ ]:
#@title Plot out the time series of EVI for each land cover type

lc_labels = {
    "11": "Bog",
    "4": "Improved grassland",
    "7": "Acid grassland",
    "12": "Inland rock",
    "1": "Deciduous woodland",
    "21": "Suburban",
    "2": "Coniferous woodland",
    "9": "Heather"
}

plt.figure(figsize=(8,5))

for lc in summary_evi["landcover"].unique():
    sub = summary_evi[summary_evi["landcover"] == lc]
    label = lc_labels.get(str(lc), str(lc))
    plt.plot(sub["date"], sub["mean_evi"], label=label)

plt.xlabel("Time")
plt.ylabel("Mean EVI")
plt.title("EVI time series by landcover")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


#Question: Which land cover type has the most variable EVI throughout the year?

# Longer-term variations in EVI using A MODIS EVI product

So far we've worked exlusivley with 30-m resolution Landsat data, which has a revisit period of 16 days.

The MODIS satellite, which has a daily revisit period collects surface reflecatance measurements that are suitable for calculating vegetation indices.

[MODIS vegetation indices](https://modis.gsfc.nasa.gov/data/dataprod/mod13.php), produced every 16-day and at multiple spatial resolutions (250 m is the finest resolution), provide consistent spatial and temporal comparisons of vegetation canopy greenness, and canopy structure.

The 16-day [MODIS EVI product](https://www.earthdata.nasa.gov/data/catalog/lpcloud-mod13q1-061) is a composite image, generated using two 8-day composite surface reflectance data (MOD09A1) in the 16-day period. The MODIS EVI product is computed from atmospherically corrected bi-directional surface reflectances that have been masked for water, clouds, heavy aerosols, and cloud shadows.

Here, will explore longer-term variations in vegetation condition using the MODIS EVI data product, which minimizes canopy-soil variations and improves sensitivity over dense vegetation conditions.

As usual, we need to first download the data and undertake some basic data processing to get it into a format that we can work with.

Note that these data are in the British National Grid projected coordinate system.

In [ ]:
#@title Download MODIS EVI data from Github


URL = "https://github.com/laura-turnbull-lloyd/IGR_remote_sensing/raw/refs/heads/main/Data/MODIS_EVI_Langdon_Beck.tif"
DOWNLOAD_DIR = "/content/downloads"
OUT_PATH = os.path.join(DOWNLOAD_DIR, "MODIS_EVI_Langdon_Beck.tif")

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download(url, out_path):
    r = requests.get(url, stream=True, allow_redirects=True)
    r.raise_for_status()

    total = int(r.headers.get("content-length", 0)) or None
    chunk = 1024 * 1024

    with open(out_path, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=os.path.basename(out_path)) as pbar:
        for part in r.iter_content(chunk_size=chunk):
            if part:
                f.write(part)
                pbar.update(len(part))

download(URL, OUT_PATH)

print("\nSaved to:", OUT_PATH)
print("Folder contents:", os.listdir(DOWNLOAD_DIR)[:20])

# Check file size
size_mb = os.path.getsize(OUT_PATH) / (1024**2)
print(f"File size: {size_mb:.2f} MB")



In [ ]:
#@title Open the multi-band GeoTIFF with rasterio

TIF_PATH = "/content/downloads/MODIS_EVI_Langdon_Beck.tif"

with rasterio.open(TIF_PATH) as src:
    crs = src.crs                   # Projection
    transform = src.transform       # Affine transform
    bounds = src.bounds             # Spatial extent
    # Band names/descriptions
    band_names = list(src.descriptions)  # may contain None entries
    band_names = [name[6:] for name in band_names] # tidy up the band names

    # Read all bands into memory (bands, rows, cols)
    MODIS_EVI = src.read().astype("float32")

# Convert nodata to NaN for nicer plotting/stats
if src.nodata is not None:
    MODIS_EVI[MODIS_EVI == src.nodata] = np.nan

print("First 10 band names:", band_names[:10])



In [ ]:
#@title Quick check that the data look okay

plt.figure(figsize=(6,6))
plt.imshow(MODIS_EVI[0],extent=[bounds.left, bounds.right, bounds.bottom, bounds.top])
plt.xlabel("Easting")
plt.ylabel("Northing")
plt.colorbar(label="Value")
plt.show()

In [ ]:
#@title In the code block below, enter the appropriate code to determine how many time slices of data are there in the MODIS_EVI dataset.

# enter your code here


##Question: How many time slices of data are there in the MODIS_EVI dataset?

Let's take a look at the data. It's a long dataset so it's useful to have a close inspection, but also to look at multiple time slices together. We can do this in multiple ways. In the code below, there are two functions: one to plot a single band of data, and another to plot many bands of data. There is also an option to plot a subset of data, such as every 12 bands (so you can view the same month across the years).

First load the functions, and then experiment with plotting different time slices of the data to get a sense of how the EVI varies over time. To do this, you will need to comment/uncomment the relevant lines using the ```#``` symbol.

In [ ]:
#@title Functions to plot a single/multiple band(s) of data.

def plot_band(idx, vmin=None, vmax=None):
    """Plot a single band index (0-based)."""
    arr = MODIS_EVI[idx]
    plt.figure(figsize=(5, 5))
    im = plt.imshow(arr, origin="upper", vmin=vmin, vmax=vmax)
    plt.title(band_names[idx])
    plt.colorbar(im, label="EVI")
    plt.axis("off")
    plt.show()

# Function to plot many bands
def plot_many(indices, ncols=6, vmin=None, vmax=None, figsize_per_panel=2.6):
    """Plot multiple bands in a grid."""
    n = len(indices)
    nrows = math.ceil(n / ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(figsize_per_panel*ncols, figsize_per_panel*nrows))
    axes = np.array(axes).reshape(-1)

# specify the min and max values to be displayed
    vmin = 0
    vmax = 1

    for k, i in enumerate(indices):
        ax = axes[k]
        im = ax.imshow(MODIS_EVI[i], origin="upper", vmin=vmin, vmax=vmax)
        ax.set_title(band_names[i], fontsize=8)
        ax.axis("off")

    for j in range(n, len(axes)):
        axes[j].axis("off")

    fig.colorbar(im, ax=axes[:n], shrink=0.75, label="EVI")
    plt.show()


In [ ]:
#@title Plotting the MODIS EVI

# 1) Plot the first band
#plot_band(0)    #***uncomment this line if you want to plot a single band

# 2) Plot the first x bands as a multipanel (adjust as you like)
# plot_many(list(range(min(12, MODIS_EVI.shape[0]))), ncols=6) #***uncomment this line if you want to plot multiple bands.
                                                               #***you can change the number 12 - this is how many bands you
                                                               #***will plot.

# 3) Plot every 10th band (nice for long time series)
step = 10  # you can change this
idx = list(range(0, MODIS_EVI.shape[0], step))
plot_many(idx[:36], ncols=6)  # show up to first 36 panels - you can change this.

Now, we'll repeat what we did before, summarizing the EVI values at each timestep by calculating the mean and median values.

In [ ]:
#@title Calculate summary statistics for the MODIS EVI values

# Convert the band names into a proper date format
dates = [pd.to_datetime(nm[-8:], format="%Y%m%d", errors="coerce") for nm in band_names]

# Compute stats for each layer (ignoring NaNs)
mean_evi_MODIS   = np.nanmean(MODIS_EVI, axis=(1, 2))
median_evi_MODIS = np.nanmedian(MODIS_EVI, axis=(1, 2))

# Build a summary table
summary_MODIS = pd.DataFrame({
    "band": np.arange(1, MODIS_EVI.shape[0] + 1),
    "layer_name": band_names,
    "date": dates,
    "mean_evi": mean_evi_MODIS.astype(float),
    "median_evi": median_evi_MODIS.astype(float),
})

# df (or df_plot) is your summary table for later use
summary_MODIS.head()


In [ ]:
#@title On your own, have a go at saving this dataframe to a csv file (you might want to use it in future!)




Now for the fun bit. Let's plot out how the mean and median EVI vary through time, from 2020 to 2025.

In [ ]:
# Plot the mean and median MODIS EVI on the same figure
fig, ax = plt.subplots(figsize=(11, 4))

ax.plot(summary_MODIS["date"], summary_MODIS["mean_evi"], marker="o", linewidth=1, label="Mean MODIS EVI", color="red")
ax.plot(summary_MODIS["date"], summary_MODIS["median_evi"], marker="o", linewidth=1, label="Median MODIS EVI", color="blue")

ax.set_xlabel("Date")
ax.set_ylabel("EVI")
ax.set_title("EVI over Langdon Beck catchment (mean and median)")
ax.grid(True)
ax.legend()

fig.tight_layout()
plt.show()



Once you've managed to show the Landsat EVI values alongside the MODIS EVI values, you can append the following to your code in order to zoom in on 2025:

```ax.set_xlim(pd.Timestamp("2025-01-01"), pd.Timestamp("2025-12-31"))```

This needs to come before
```fig.tight_layout()``` and ```plt.show()```


# Time to reflect

Clearly there is a trade-off between the high-resolution and less frequent Landsat data and the lower resolution, high frequency MODIS data.

If your research goal is to be able to make statements about how vegetation condition (or some other surface characteristic) varies over time, in accordance with land use, or something else such as soil type, aspect, or slope, then you might want to consider the scale at which these properties vary. Can the variability in the structure of the landscape be adequately represented by the resolution of the satellite imagery? Here, whether or not it's adequate will depend on your research question.

A 250 x 250 m MODIS cell is actually quite large, so in a heterogeneous landscape, the surface reflectance value for a single pixel is going to be an aggregation of smaller scale variability.

Despite these limitations, the high acquisition frequency is advantageous, as it reduces the influence of cloud-related data gaps.

What is impressive is the high level of agreement between the EVI values measured by the two different sensors. The band widths are actually slightly different (i.e. they're recording slightly different things), but the overall impacts on the resulting EVI is clearly very small.

